In [16]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [17]:
pip install caer

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.5/809.5 KB 297.9 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 677.3 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 730.8 kB/s eta 0:00:00m eta 0:00:010:00:03
Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install caer canaro

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import matplotlib.pyplot as plt
import os
import gc

import cv2 as cv
import caer
import canaro
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

2024-02-10 16:51:57.667384: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-10 16:51:57.667477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-10 16:51:57.712223: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
img_size=(180,180)
channels = 1
char_path= 'Data/simpsons_dataset'

In [3]:
char_dict={}
for char in os.listdir(char_path):
    char_dict[char]=len(os.listdir(os.path.join(char_path,char)))
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

[('homer_simpson', 2246),
 ('ned_flanders', 1454),
 ('moe_szyslak', 1452),
 ('lisa_simpson', 1354),
 ('bart_simpson', 1342),
 ('marge_simpson', 1291),
 ('krusty_the_clown', 1206),
 ('principal_skinner', 1194),
 ('charles_montgomery_burns', 1193),
 ('milhouse_van_houten', 1079),
 ('chief_wiggum', 986),
 ('abraham_grampa_simpson', 913),
 ('sideshow_bob', 877),
 ('apu_nahasapeemapetilon', 623),
 ('kent_brockman', 498),
 ('comic_book_guy', 469),
 ('edna_krabappel', 457),
 ('nelson_muntz', 358),
 ('lenny_leonard', 310),
 ('mayor_quimby', 246),
 ('waylon_smithers', 181),
 ('maggie_simpson', 128),
 ('groundskeeper_willie', 121),
 ('barney_gumble', 106),
 ('selma_bouvier', 103),
 ('carl_carlson', 98),
 ('ralph_wiggum', 89),
 ('patty_bouvier', 72),
 ('martin_prince', 71),
 ('professor_john_frink', 65),
 ('snake_jailbird', 55),
 ('cletus_spuckler', 47),
 ('rainier_wolfcastle', 45),
 ('simpsons_dataset', 42),
 ('agnes_skinner', 42),
 ('sideshow_mel', 40),
 ('otto_mann', 32),
 ('gil', 27),
 ('fat_

In [4]:
chars = []
cnt=0
for i in char_dict:
    chars.append(i[0])
    cnt+=1
    if cnt>=10:
        break
chars

['homer_simpson',
 'ned_flanders',
 'moe_szyslak',
 'lisa_simpson',
 'bart_simpson',
 'marge_simpson',
 'krusty_the_clown',
 'principal_skinner',
 'charles_montgomery_burns',
 'milhouse_van_houten']

In [5]:
#Создание тренировочных данных
train=caer.preprocess_from_dir(char_path, chars, channels=channels, IMG_SIZE=img_size, isShuffle=True)

[INFO] Could not find a file to load from. Generating the training data
----------------------------------------------
[INFO] At 1000 files
[INFO] At 2000 files
[INFO] 2246 files found in 0.003773927688598633s
[INFO] At 1000 files
[INFO] 1454 files found in 0.0018842220306396484s
[INFO] At 1000 files
[INFO] 1452 files found in 0.0018596649169921875s
[INFO] At 1000 files
[INFO] 1354 files found in 0.0017323493957519531s
[INFO] At 1000 files
[INFO] 1342 files found in 0.0017592906951904297s
[INFO] At 1000 files
[INFO] 1291 files found in 0.001683950424194336s
[INFO] At 1000 files
[INFO] 1206 files found in 0.0015919208526611328s
[INFO] At 1000 files
[INFO] 1194 files found in 0.0015728473663330078s
[INFO] At 1000 files
[INFO] 1193 files found in 0.0015637874603271484s
[INFO] At 1000 files
[INFO] 1079 files found in 0.0014498233795166016s
----------------------------------------------
[INFO] 13811 files preprocessed! Took 0m 22s


In [6]:
# len(train)

In [7]:
# plt.figure(figsize=(1,1))
# plt.imshow(train[0][0], cmap='gray')
# plt.show

In [8]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=img_size)

In [9]:
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(chars))

In [10]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=0.1)

In [12]:
# x_train.shape
# y_train.shape

In [14]:
# Создаем модель нейронной сети
model = Sequential()

# Добавляем сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(180, 180, 1)))

# Добавляем слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Добавляем второй сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))

# Добавляем второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Преобразуем двумерные данные в одномерные
model.add(Flatten())

# Добавляем полносвязный слой
model.add(Dense(64, activation='relu'))

# Добавляем выходной слой
model.add(Dense(10, activation='softmax'))

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.00001, decay=1e-7, momentum=0.7, nesterov=True)

# Компилируем модель
model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 178, 178, 64)      640       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 89, 89, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 87, 87, 64)        36928     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 118336)            0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [15]:
del train
del featureSet
del labels
gc.collect()

1298

In [16]:
BATCH_SIZE=10
EPOCHS=25

In [17]:
datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1
                             , zoom_range=0.2)
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [18]:
callbacks_list=[LearningRateScheduler(canaro.lr_schedule)]

In [19]:
# optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.00001, decay=1e-7, momentum=0.7, nesterov=True)
# len(x_train)//10
len(y_val)//10

138

In [21]:
training=model.fit(train_gen, steps_per_epoch=len(x_train)//BATCH_SIZE, epochs=EPOCHS, validation_data=(x_val, y_val),
                   validation_steps=len(y_val)//BATCH_SIZE, callbacks=callbacks_list)

Epoch 1/25
1242/1242 [==============================] - 256s 206ms/step - loss: 0.3251 - accuracy: 0.1565 - val_loss: 0.3218 - val_accuracy: 0.1858 - lr: 0.0100
Epoch 2/25
1242/1242 [==============================] - 242s 195ms/step - loss: 0.3130 - accuracy: 0.2057 - lr: 0.0100
Epoch 3/25
1242/1242 [==============================] - 256s 206ms/step - loss: 0.3044 - accuracy: 0.2499 - lr: 0.0100
Epoch 4/25
1242/1242 [==============================] - 250s 201ms/step - loss: 0.2998 - accuracy: 0.2681 - lr: 0.0100
Epoch 5/25
1242/1242 [==============================] - 255s 205ms/step - loss: 0.2959 - accuracy: 0.2827 - lr: 0.0100
Epoch 6/25
1082/1242 [=========================>....] - ETA: 32s - loss: 0.2927 - accuracy: 0.2931

KeyboardInterrupt: 

In [78]:
val_datagen = canaro.generators.imageDataGenerator()
val_gen = val_datagen.flow(x_val, y_val, batch_size=BATCH_SIZE)
model.evaluate(val_gen)

1/1 [==============================] - 0s 224ms/step - loss: 0.3769 - accuracy: 1.0000


[0.37694883346557617, 1.0]

In [96]:
img1 = cv.imread('Data/kaggle_simpson_testset/kaggle_simpson_testset/ned_flanders_12.jpg')
def prepare(image, img_size):
    image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    image = (image/10).astype('uint8')
    image = cv.resize(image, img_size)
    image = caer.reshape(image, img_size, 1)
    return image

In [97]:
img_size = (180,180)
predictions = model.predict(prepare(img1, img_size))

1/1 [==============================] - 0s 37ms/step


In [98]:
print(chars[np.argmax(predictions[0])])

homer_simpson
